# Practical 1: Power Balancing for a Local Community

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import functions_T1
import simulation_creator_T1_Balancing

ModuleNotFoundError: No module named 'simulation_creator_T1_Balancing'

A neighbourhood is connected to the grid as sketched below. For a typical day in the year the neighbourhood has a given demand profile. To cover this demand electricity is taken from the grid.

**Task 1**: The following python code simulates a typical day based on hourly demand data. Run the code and describe what is seen in the plot. (maybe include quick description of what the python code does)

![T1_basecase](Graphics_T1/T1_basecase.png)

In [ ]:
load_df = pd.read_csv('../Scenarios/load_data.txt', delimiter=',', skiprows=1)
load_df['time'] = pd.to_datetime(load_df['time'])
load_df.set_index('time', inplace=True)

#choose which day of the year to plot
day_of_year = "2012-06-01"
plot_load_base_case = functions_T1.plot_load_profile(load_df, day_of_year)

AttributeError: module 'datetime' has no attribute 'strptime'

The community implements small scale PV and wind power plants to become more autonomous. Now first the supply from the local renewable energy sources is used to cover the demand in the community. The residual load is then supplied by the grid.

![T1_RES_case](Graphics_T1/T1_RES_case.png)

**Task 2**: Run the following python code and look at the resulting residual load plot. Describe the plot and compare it to the results from task 1. -> include size variations

In [ ]:
day_of_year = "2012-06-01"

#Component Choice
Components_RES_case = pd.DataFrame()     # moved to notebook to choose what is included and what not in set up -> need to adjust in simulation set up
Components_RES_case['model'] = pd.Series(['Wind', 'PV', 'Load'])

#Input Data Simulation
pv_panel_set ={'Module_area': 1.26, 'NOCT': 44, 'Module_Efficiency': 0.198, 'Irradiance_at_NOCT': 800,  #panel specs
          'Power_output_at_STC': 250,'peak_power':600}
pv_set={'m_tilt':14,'m_az':180,'cap':500,'output_type':'power'}

load_set={'houses':5, 'output_type':'power'}    #demand specs

Wind_set={'p_rated':300, 'u_rated':10.3, 'u_cutin':2.8, 'u_cutout':25, 'cp':0.40, 'diameter':22, 'output_type':'power'} #wind specs



The residents have decided to implement a community storage, where the electricity generated from the local renewable energy sources is stored when generation exceeds the local demand. The stored energy is used when the local generation cannot cover the demand. (+ Explanation SoC parameter in python script)

![T1_RES_case_Bat](Graphics_T1/T1_RES_case_Bat.png)

**Task 3a)**: Run the following python code that simulates a typical day in the local community with renewable generation and battery storage. For this task the State of charge is 100% in the beginning of the day. The controller acts in the following way: in the case of not enough local generation to cover the demand of the neighbourhood electricity is taken from the battery storage if possible, if this is not possible electricity is taken from the grid. How does the residual load curve change compared to task 2 (the utilization of small scale local renewable generation without storage).

**Task 3b)**: Change the State of charge parameter to 50% in the beginning of the day and run the simulation again. Explain how this affects the residual load curve.

**Task 3c)**: Change the State of charge parameter to 10% in the beginning of the day and run the simulation again. Explain how this affects the residual load curve.

Note: Changing Objective Controller: Contractual/Physical Limit for connection (Peak shaving & Load Shifting, Load Shedding)
EV/Flexibilities

In [ ]:
day_of_year = "2012-06-01"

Components_RES_Bat_case = pd.DataFrame()     # moved to notebook to choose what is included and what not in set up -> need to adjust in simulation setup
Components_RES_Bat_case['model'] = pd.Series(['Wind', 'PV', 'Load', 'Battery'])

#Input Data Simulation
pv_panel_set ={'Module_area': 1.26, 'NOCT': 44, 'Module_Efficiency': 0.198, 'Irradiance_at_NOCT': 800,  #panel specs -> maybe adjust so number of modules can be changed
          'Power_output_at_STC': 250,'peak_power':600}
pv_set={'m_tilt':14,'m_az':180,'cap':500,'output_type':'power'}

load_set={'houses':5, 'output_type':'power'}    #demand specs

Wind_set={'p_rated':300, 'u_rated':10.3, 'u_cutin':2.8, 'u_cutout':25, 'cp':0.40, 'diameter':22, 'output_type':'power'} #wind specs

Battery_initialset = {'initial_soc': 20} # combine to battery set
Battery_set = {'max_p': 800, 'min_p': -800, 'max_energy': 800,
            'charge_efficiency': 0.9, 'discharge_efficiency': 0.9,
            'soc_min': 10, 'soc_max': 90, 'flag': 0,'resolution':15}

